# PyMetaSeem
CAMISIM es un `Programa` para simular datos metagenomicos a partir de genomas, a grandes rasgos este programa toma uno o varios archivos de genomas, y por un proceso de corte aleatorio de secuencias crea un archivo de datos metagenomicos.

A continuación se muestran los pasos para hacer una recreaccion de este algoritmo, y explicando cada una de las funciones creadas.

In [1]:
import os
import math
import gzip
import random
import numpy as np

from pathlib import Path
from random import seed
from random import randint

Antes de empesar a usar PyMetaSeem, asegurese que sus archivos Fasta en la primera linea de 

In [ ]:
# Bash

# vi
#:/s

La primera funcion se encarga de leer uno o varios genomas en archivos, los cuales se convertiran en un diccionario para poder trabajar con ellos en `Python`, en donde las `llaves`son los identificadores de los genoma y los `valores` la secuencia de nucleotidos.

In [2]:
# funcion para lectura de archivos
# ingresa un archivo multifasta y arroja una lista con cada secuencia (contig)
dicc_contigs = {}
def lectura_genoma(File):
    
    lista = []
    longitudes = []
    with open(File,'r') as f:
        lines=f.read() # lectura de cada linea del archivo
        lines=lines.split('>') # identificador de '>'
        lines=['>'+ x for x in lines[1:]] # lista con cada elemento que comienza con '>'
        
        for x in lines:
            x1 = x.replace(">","@")
            x2 = x1.replace("\n",",",1) # el primer '\n' se reemplazapor una coma
            x3 = x2.replace("\n","") # los siguientes se quitan
            lista.append(x3) # lista con un contig en cada elemento y su identificador
    
        # convertir la lista en un diccionario        
        for x in lista:
            x = x.split(',')
            dicc_contigs[x[0]] = x[1] 
        return(dicc_contigs)    

In [3]:
dicc_contigs = lectura_genoma('/home/csilva/GIT/Tesis_Maestria/Data/all-c-genomes(cortados)/GenomaPrueba.fna')

In [4]:
# funcion que cuenta los contigs de cada genoma
def cuenta_contigs(dicc_contigs):
    long_total = 0
    dicc_longitudes = {}
    num_contigs = len(dicc_contigs) # cuenta cuantos contigs contiene cada genoma (cuantas llaves)
    for key in dicc_contigs:
        longitud = len(dicc_contigs[key])# calcula la longitud de cada contig (la longitud de las claves, para cada llave)
        long_total +=  longitud # va sumando todas las longitudes de los contigs
        dicc_longitudes[key] = longitud # diccionario de longitudes 
        # key es el identificador del contig 
        # longitud -> es la longitud de cada contig
        # long_total -> lontitud total del genoma
    return (dicc_longitudes,num_contigs,long_total)   
        
        
        
        
### SE TOMA EL MINIMO DE CANTIDAD DE CONTIGS (se debe comparar por cada genoma)
### TOMAR EL NUMERO MINIMO DE CONTIGS CON MAYOR LONGITUD

In [5]:
dicc_longitudes,num_contigs,long_total = cuenta_contigs(dicc_contigs)

In [6]:
dicc_longitudes,num_contigs,long_total

({'@NZ_CP048049.1': 160, '@NZ_CP021034.1': 171, '@NZ_CP086349.1': 85}, 3, 416)

In [7]:
# funcion para calcular la proporcion de los contigs
def proporcion_contigs(dicc_contigs,dicc_longitudes,long_total):
    dicc_proporciones = {}
    for key in dicc_longitudes:
        P = dicc_longitudes[key]/long_total
        dicc_proporciones[key] = P
    return dicc_proporciones
# numero de reads por contig se calcula en la funcion reads 

In [8]:
dicc_proporciones = proporcion_contigs(dicc_contigs,dicc_longitudes,long_total)

In [9]:
dicc_proporciones

{'@NZ_CP048049.1': 0.38461538461538464,
 '@NZ_CP021034.1': 0.4110576923076923,
 '@NZ_CP086349.1': 0.20432692307692307}

Esta funcion se encarga de cortar una secuencia de nucleotidos dada, a partir de una pocision y una longitud dada.

In [10]:
# funcion para cortar reads, dado una pocision de inicio y una longitud de corte
def cutout(contig,i,n_length): #fordward    (resive contig y entrega reads)
    read = contig[i:i+n_length]
    return(read) 

In [11]:
cutout(dicc_contigs['@NZ_CP048049.1'],5,10)

'CGACCGCTCC'

### Reads reverse

In [12]:
def cutout2(contig,i,n_length,inserto): #backward 
    # inserto = 400 (tamaño promedio del inserto), depende del secuenciador y lo da el usuario
    # inserto -> tamaño verdadero de la molecula de DNA que esta en el secuanciador
    j = i + inserto
    # j = i+inserto (pocision de inicio de reversa)
    # n_length = 150 (tamaño del read "cortado")
    read_cortado = contig[j-n_length:j]
    #i_read = contig[j-n_length:inserto]  #(quiero comenzar en la pocision j e ir hacia atras)
    return read_cortado

In [13]:
cutout('ACTTGTAACTTGTAACTTGTAACTTGTAACTTGTATTGTATTGTAACTTG',5,10)

'TAACTTGTAA'

In [14]:
cutout2('ACTTGTAACTTGTAACTTGTAACTTGTAACTTGTATTGTATTGTAACTTG',5,10,30)   # una funcion de solo corte y otra de solo reverse

'GTAACTTGTA'

In [15]:
# funcion que calcula el reverso de un Read 
def reverso(read_cortado):
    temp_list = list(read_cortado)
    temp_list.reverse()
    return ''.join(temp_list)

In [16]:
reverso('GTAACTTGTA')  

'ATGTTCAATG'

In [17]:
def complementario(read_cortadoR):
    comp = []
    for i in range(len(read_cortadoR)):
        if read_cortadoR[i] == "A":
            comp.append("T")
        elif read_cortadoR[i] == "T":
            comp.append("A")
        elif read_cortadoR[i] == "G":
            comp.append("C")
        elif read_cortadoR[i] == "C":
            comp.append("G") 
            
    return "".join(comp)

In [18]:
complementario('ATGTTCAATG') 

'TACAAGTTAC'

In [19]:
def reverso_complementario(contig,i,n_length,inserto):
    read_rev_com = complementario(reverso(cutout2(contig,i,n_length,inserto))) 
    return (read_rev_com) # entrega cada read reverso complementario

In [20]:
reverso_complementario('ACTTGTAACTTGTAACTTGTAACTTGTAACTTGTATTGTATTGTAACTTG',5,10,30)
# si se pone este read de ejemplo, el reverso complementario deberia quedar 'TAC'

'TACAAGTTAC'

Esta función se depende de la funcion anterior para cortar reads, y se complementa haciendo esto para cada genoma dentro de diccionario creado.

In [32]:
# funcion para cortar los reads aleatoriamente
# Al ingresar un conjunto de genomas, este nos arroja un metagenoma con varios reads tomados de los genomas iniciales
# para esto se le entrega el numero de reads deseados por cada genoma y la longitud.

def reads(dicc_contigs,dicc_longitudes,dicc_proporciones,n_length,num_reads_total):  
    
    dicc_reads = {}  # diccionario de reads 
    dicc_reads_reverse = {} # diccionario de reverso reads
    k = 0
    kk = 0
    inserto = 30 #400 #(tamaño promedio del inserto), depende del secuenciador y lo da el usuario
     
    # LONGITUD Y NUM_READS DEPENDERA DE LAS LONGITUDES DE LOS CONTIG
    # LA LONGITUD 
    # LAS LONGITUDES DEPENDEN DE LA CALIDAD
    
    while k < num_reads_total: # número de reads total
              
        for key in dicc_contigs:
            #contig = random.choice(list(dicc_contigs.values())) # escoje un contig al azar
            num_reads_contig = round(num_reads_total* dicc_proporciones[key]) # numero de reads x proporcion #proporcion de reads por contig
            contig = dicc_contigs[key]
            # la proporcion viene del diccionario proporciones de la funcion cuenta contigs 
            for kk in range(num_reads_contig): 
                newkey = key + '_' + str(kk) # identificador de contig, con el numero de read
                i = randint(1,(len(contig)-n_length-inserto-1)) # toma una pocision de inicio al azar se debe tener en cuenta el inserto)
                dicc_reads[newkey] = cutout(contig,i,n_length) # para el contig dado anteriormente, se corta desde la pocision i de lonjitud ln
                dicc_reads_reverse[newkey] = reverso_complementario(contig,i,n_length,inserto)# reverso complementario

                kk += 1
            k += num_reads_contig # funcion para cortar los reads aleatoriamente
        # se debe calcular cuantas veces va a escoger cada contig,dependiendo del tamaño del contig inicial      

    return(dicc_reads,dicc_reads_reverse)

In [33]:
dicc_reads,dicc_reads_reverse = reads(dicc_contigs,dicc_longitudes,dicc_proporciones,5,10)

In [34]:
dicc_reads_reverse

{'@NZ_CP048049.1_0': 'GATCG',
 '@NZ_CP048049.1_1': 'ATGCG',
 '@NZ_CP048049.1_2': 'CACCT',
 '@NZ_CP048049.1_3': 'CCTTC',
 '@NZ_CP021034.1_0': 'CATCA',
 '@NZ_CP021034.1_1': 'TCACG',
 '@NZ_CP021034.1_2': 'CGGGG',
 '@NZ_CP021034.1_3': 'GTGAG',
 '@NZ_CP086349.1_0': 'TGAGG',
 '@NZ_CP086349.1_1': 'GATCG'}

In [35]:
dicc_reads

{'@NZ_CP048049.1_0': 'TCCGA',
 '@NZ_CP048049.1_1': 'GCTGT',
 '@NZ_CP048049.1_2': 'GTCAT',
 '@NZ_CP048049.1_3': 'CCGAC',
 '@NZ_CP021034.1_0': 'ATCAG',
 '@NZ_CP021034.1_1': 'TCATC',
 '@NZ_CP021034.1_2': 'GCCCT',
 '@NZ_CP021034.1_3': 'GATCT',
 '@NZ_CP086349.1_0': 'CGATC',
 '@NZ_CP086349.1_1': 'TCCGA'}

In [ ]:
# funcion para simular la calidad

La ultima función toma el diccionario de reads creado por la anterior funcion, y lo guarda en un archivo `.fastq` , creando una calidad para cada read.

In [52]:
# funcion de creacion archivo .fastq por cada genoma
def crea_fastq(dicc_reads,dicc_reads_reverso,n_length,file_name):
    file = open(file_name+'_R1.fastq.gz','wt') 
    for key in dicc_reads: 
        file.write(str(key))
        file.write(str('\n'))
        file.write(str(dicc_reads[key]))
        file.write(str('\n'))
        file.write(str('+'))
        file.write(str('\n'))
        file.write(str('A'*n_length)) #calidad
        file.write(str('\n'))     
    # agrega el reverso complementario  
    file = open(file_name+'_R2.fastq.gz','wt')
    for key in dicc_reads_reverse: 
        file.write(str(key))
        file.write(str('\n'))
        file.write(str(dicc_reads_reverso[key]))
        file.write(str('\n'))
        file.write(str('+'))
        file.write(str('\n'))
        file.write(str('A'*n_length)) #calidad
        file.write(str('\n'))  
    file.close()    

In [53]:
crea_fastq(dicc_reads,dicc_reads_reverso,10,"prueba")

probar con archivos comprimidos .gzip

In [54]:
# funcion de creacion archivo .fastq.gz por cada genoma
def crea_fastqz(dicc_reads,dicc_reads_reverse,n_length,file_name):
    file = gzip.open(file_name+'_R1.fastq.gz','wt')
    for key in dicc_reads: 
        file.write(str(key))
        file.write(str('\n'))
        file.write(str(dicc_reads[key]))
        file.write(str('\n'))
        file.write(str('+'))
        file.write(str('\n'))
        file.write(str('A'*n_length)) #calidad
        file.write(str('\n'))     
    # agrega el reverso complementario  
    file = open(file_name+'_R2.fastq.gz','wt')
    for key in dicc_reads_reverse: 
        file.write(str(key))
        file.write(str('\n'))
        file.write(str(dicc_reads_reverse[key]))
        file.write(str('\n'))
        file.write(str('+'))
        file.write(str('\n'))
        file.write(str('A'*n_length)) #calidad
        file.write(str('\n'))  
    file.close()

y aplicando lo visto en clase sobre lectura de arhivos, e iteracion sobre cada uno de ellos.

In [ ]:
# Funcion para la lectura de los nombres de los todos los archivos de genomas
# toma la ruta de la carpeta donde estan los genomas, y crea un listado de los nombres de los archivos
# para poder realizar la lectura de los mismos.
path = '/home/csilva/GIT/Tesis_Maestria/Data/all-c-genomes(cortados)'
contenido = os.listdir(ruta)
genomes = []
for i in contenido:Search or jump to…
    if os.path.isfile(os.path.join(ruta,i)) and i.endswith('.fna'):
        genomes.append(i)
n = len(genomes) # número de genomas (número de archivos)
diccionario_reads_metagenoma = {}
    

In [ ]:
#genomes

Funcion para leer los genomas en una carpeta

In [ ]:
# Con esta funcion queremos llamar a las funciones de la clase,para todos los archivos en la lista 'genomes'

def todo(genomes,longitud_read,num_reads,file_name):
    for i,x in enumerate(genomes):
        dicc_contigs = lectura_genoma(path + genoma)
        dicc_longitudes,num_contigs,long_total = cuenta_contigs(dicc_contigs)
        dicc_proporciones = proporcion_contigs(dicc_contigs,dicc_longitudes,long_total)
        dicc_reads,dicc_reads_reverso = reads(dicc_contigs,dicc_longitudes,dicc_proporciones,longitud_read,num_reads)
        crea_fastq(dicc_reads,dicc_reads_reverso,longitud_read,name)
    

In [ ]:
todo(genomes,150,5,'prueba_clase.fastq')

Funcion para leer los genomas en una carpeta por medio de un archivo .csv,en donde se encuentra lainformacion necesaria 

In [ ]:
path = '/home/betterlab/shaday/ReadsFonty/cut_head/'
def todo(genomes, longitud_read,num_reads,name):
    dicc_contigs = lectura_genoma(path + genoma)
    dicc_longitudes,num_contigs,long_total = cuenta_contigs(dicc_contigs)
    dicc_proporciones = proporcion_contigs(dicc_contigs,dicc_longitudes,long_total)
    dicc_reads,dicc_reads_reverso = reads(dicc_contigs,dicc_longitudes,dicc_proporciones,longitud_read,num_reads)
    crea_fastq(dicc_reads,dicc_reads_reverso,longitud_read,name)

In [ ]:
dp= pd.read_csv(path + 'table.txt', sep='\t')
dp

In [ ]:
for i in range(len(dp.iloc[:,4])):
    todo(dp.iloc[i,4],150,dp.iloc[i,3],dp.iloc[i,1])

Para Shaday:
* 9 archivos  

1. (3 de un solo genoma)
    * 1.1
        1. Clavibacter_michiganensis_subsp_capsici_PF008.fna-cortado.fna

    * 1.2
        1. Clavibacter_michiganensis_subsp_nebraskensis_CIBA.fna-cortado.fna

    * 1.3
        1. Clavibacter_michiganensis_subsp_sepedonicus_CFIA-Cs3N.fna-cortado.fna
        
2. (3 de 2) 
    * 2.1
        1. Clavibacter_michiganensis_subsp_capsici_1106.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_nebraskensis_7580.fna-cortado.fna
        
    * 2.2
        1. Clavibacter_michiganensis_subsp_insidiosus_LMG_3663.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado.fna
        
    * 2.3
        1. Clavibacter_michiganensis_subsp_sepedonicus_ATCC33113.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado

3. (3 de 3)
    * 3.1
        1. Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_nebraskensis_CFBP_7577.fna-cortado.fna
        3. Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna  
        
    * 3.2
        1. Clavibacter_michiganensis_subsp_insidiosus_CFBP_6488.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna   
        3. Clavibacter_michiganensis_subsp_tessellarius_ATCC33113.fna-cortado.fna
        
    * 3.3
        1. Clavibacter_michiganensis_subsp_insidiosus_CFBP_2404.fna-cortado.fna
        2. Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado.fna
        3. Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna  


In [ ]:
# 3 listas con 1 genoma
genomes11 = ['Clavibacter_michiganensis_subsp_capsici_PF008.fna-cortado.fna']
genomes12 = ['Clavibacter_michiganensis_subsp_nebraskensis_CIBA.fna-cortado.fna']
genomes13 = ['Clavibacter_michiganensis_subsp_sepedonicus_CFIA-Cs3N.fna-cortado.fna']
# 3 listas con 2 genomas
genomes21 = ['Clavibacter_michiganensis_subsp_capsici_1106.fna-cortado.fna','Clavibacter_michiganensis_subsp_nebraskensis_7580.fna-cortado.fna']
genomes22 = ['Clavibacter_michiganensis_subsp_insidiosus_LMG_3663.fna-cortado.fna','Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado.fna']
genomes23 = ['Clavibacter_michiganensis_subsp_sepedonicus_ATCC33113.fna-cortado.fna','Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado']
# 3 listas con 3 genomas
genomes31 = ['Clavibacter_michiganensis_subsp_capsici_1101.fna-cortado.fna','Clavibacter_michiganensis_subsp_nebraskensis_CFBP_7577.fna-cortado.fna','Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna']
genomes32 = ['Clavibacter_michiganensis_subsp_insidiosus_CFBP_6488.fna-cortado.fna','Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna','Clavibacter_michiganensis_subsp_tessellarius_ATCC33113.fna-cortado.fna']
genomes33 = ['Clavibacter_michiganensis_subsp_insidiosus_CFBP_2404.fna-cortado.fna','Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna-cortado.fna','Clavibacter_michiganensis_subsp_sepedonicus_CFIA-CsR14.fna-cortado.fna'] 